# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import warnings
import math
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# Number of callbacks for black-sounding names
print(sum(data[data.race=='b'].call))

# Number of callbacks for white-sounding names
print(sum(data[data.race=='w'].call))

157.0
235.0


In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


Question 1: What test is appropriate for this problem? Does CLT apply?
A two-sample t-test is appropriate for comparing two independent samples with a large sample size. Because of the large sample size, the Central Limit Theorem can also be applied.

In [5]:
# Assignment of data and variables
test_bl = data.call[data.race =='b']
test_wh = data.call[data.race =='w']

bl_samp_mean = np.mean(test_bl) 
wh_samp_mean = np.mean(test_wh)

bl_n = len(test_bl)
wh_n = len(test_wh)

print('The callback rate for names that sound black is: ', bl_samp_mean)
print('The callback rate for names that sound white is: ', wh_samp_mean)

print(bl_n, wh_n)

The callback rate for names that sound black is:  0.0644763857126236
The callback rate for names that sound white is:  0.09650924056768417
2435 2435


Question 2: What are the null and alternate hypotheses? The applicable hypothesis test is H0: There is no significant difference in callbacks (bl sample mean = wh sample mean) H0: There is significant difference in callbacks (bl sample mean does not equal wh sample mean).

In [6]:
race_ttest, race_pval = stats.ttest_ind(a=test_bl, b=test_wh, equal_var=False)
print('The T-Statistic is %d with a p-value of %s' %(abs(race_ttest), race_pval))
print(race_ttest)

The T-Statistic is 4 with a p-value of 3.942941513645935e-05
-4.114705290861751


Assuming a 95% confidence level for the test we will reject the null hypothesis if the test statistic > 1.96 standard deviations and has a large p-value. Based on the results, the test statistic is 4.11 with a very small p-value. Therefore, we reject the null hypothesis at the 95% confidence interval in favor of the alternative hypothesis.

Question 3: Compute margin of error, confidence interval, and p-value. 

In [7]:
# P-value
print('The p-value is: ',(race_pval))

# Margin of error - standard error
print('The standard error in the black name sample is: ', stats.sem(test_bl))
print('The standard error in the white name sample is: ', stats.sem(test_wh))

# Confidence interval
conf_int_bl = stats.t.interval(alpha = .95, df = bl_n-1, loc = bl_samp_mean,
                 scale=stats.sem(test_bl))
print('At the 95% level, the lower bound of our confidence interval is ',conf_int_bl[0], 
      ' and the upper bound is ',conf_int_bl[1]) 

conf_int_wh = stats.t.interval(alpha = .95, df = wh_n-1, loc = wh_samp_mean,
                 scale=stats.sem(test_wh))
print('At the 95% level, the lower bound of our confidence interval is ',conf_int_wh[0], 
      ' and the upper bound is ',conf_int_wh[1])

The p-value is:  3.942941513645935e-05
The standard error in the black name sample is:  0.0049781434352911685
The standard error in the white name sample is:  0.005985301397503016
At the 95% level, the lower bound of our confidence interval is  0.054714549604450616  and the upper bound is  0.07423822182079658
At the 95% level, the lower bound of our confidence interval is  0.08477242903143205  and the upper bound is  0.1082460521039363


Question 4: What is the story describing the statistical significance in the context of the original problem.

Based on the results, there is evidence showing that there are differences in responses for black sounding names and white sounding names. The hypothesis test indicates that the mean callback rates would be the same at 4.11 standard deviations. The standard error of both samples is ~0.5% which indicates distributions around the means. In addition, no overlap appears to exist in the confidence intervals for each sample. It is unlikely that the callback rate is the same for both samples.

Question 5: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, there are other variables worth exploring to see if there are any effects on callbacks. Education and experience are most likely relavant factors for additional analysis. 